# Advent of Code 2022
## Day 7

In [43]:
import time

import pytest
import ipytest
from pprint import pprint
import math

ipytest.autoconfig()

In [44]:
def readData(filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
            f.close()
        return lines

data = readData('./data/day7.txt')
data = [line.strip('\n') for line in data]
pprint(data[:30])

['$ cd /',
 '$ ls',
 'dir fpljqj',
 '171526 ghtzhjwf.nls',
 'dir gsdsbld',
 'dir hbmjtb',
 '296801 mjfjqw.ccv',
 'dir nfn',
 'dir qmrsvfvw',
 '102565 qqjnqb.chd',
 'dir svgbqd',
 '$ cd fpljqj',
 '$ ls',
 '153563 ghtzhjwf.nls',
 '243252 gsvjgj.jsm',
 '154134 hghnrbqg.rzb',
 '$ cd ..',
 '$ cd gsdsbld',
 '$ ls',
 'dir npmncvhh',
 'dir qmrsvfvw',
 'dir sqtnlr',
 'dir vzndpc',
 '$ cd npmncvhh',
 '$ ls',
 '81366 dwbgr.ztr',
 '144577 fzjmcq',
 'dir mphhrqf',
 'dir rnmvggfd',
 '276454 zfl.ghv']


In [75]:
class directory:
    """
    Class to define a directory
    """
    def __init__(self, is_root = False, parent = None, name = None):
        self.is_root = is_root
        self.name = name
        self.dirs = {}
        self.files = set()
        self.parent = parent
        self.size = 0
        self.total = 0

    def recalculateSize(self):
        # calculates the size of the directory based on the files (excluding subdirectories)
        self.size = 0
        for file in self.files:
            self.size += file[0]
        return

    def addSubdirSize(self):
        # add the size of subdirectories to the size
        self.recalculateSize()
        for d in self.dirs.keys():
            self.size += self.dirs[d].addSubdirSize()
        return self.size

    def sumUnder100k(self):
        # recursive call to sum all directories under 100,000 in size
        self.total = 0
        for d in self.dirs.keys():
            if self.dirs[d].size <= 100000:
                self.total += self.dirs[d].size + self.dirs[d].sumUnder100k()
            else:
                self.total += self.dirs[d].sumUnder100k()
        return self.total

    def findSmallest(self, needed, smallest):
        # recursively find the size of the smallest directory that meets the requirements
        for d in self.dirs.keys():
            new = self.dirs[d].findSmallest(needed, smallest)
            if new < smallest:
                smallest = new
        if needed < self.size < smallest:
            smallest = self.size
        return smallest


In [ ]:
class fileSystem:
    def __init__(self, capacity = 70000000):
        self.root = directory(is_root = True, name = '/')
        self.current_dir = self.root
        self.capacity = capacity

    def mkDir(self, dir_name):
        # instantiate a new directory
        if dir_name not in self.current_dir.dirs.keys():
            self.current_dir.dirs[dir_name] = directory(is_root = False, parent = self.current_dir, name = dir_name)
        return

    def addFile(self, file):
        # add a file to the current dir
        self.current_dir.files.add((int(file[0]), file[1]))
        self.current_dir.size += int(file[0])
        return

    def isCommand(self, line):
        # process a line on whether it is a command
        first_char = line[0]
        if first_char == '$':
            return True
        return False

    def processCommand(self, line):
        # process the type of command (do something if it is 'cd', else do nothing)
        command_parts = line.split(' ')
        if command_parts[1] == 'cd':
            self.cd(command_parts[2])
        return

    def cd(self, direc):
        # process the change directory command and its different cases
        if direc == '..':
            self.current_dir = self.current_dir.parent
        elif direc == '/':
            self.current_dir = self.root
        else:
            self.mkDir(direc)
            self.current_dir = self.current_dir.dirs[direc]

    def processFile(self, file):
        # process the file line
        file_parts = file.split(' ')
        if file_parts[0] == 'dir' and file_parts[1] not in self.current_dir.dirs.keys():
            self.current_dir.dirs[file_parts[1]] = directory(is_root = False,
                                                             parent = self.current_dir,
                                                             name = file_parts[1])
        else:
            self.addFile(file_parts)
        return

    def buildTree(self, lines):
        # take input lines and build the file directory tree
        counter = 0
        num_lines = len(lines)
        while counter < num_lines:
            if self.isCommand(lines[counter]):
                self.processCommand(lines[counter])
            else:
                self.processFile(lines[counter])
            counter += 1
        self.root.addSubdirSize()

    def getUnder100k(self):
        # get the sum of all directories under 100,000 in size
        return self.root.sumUnder100k()

    def findMoreNeeded(self, needed_space):
        # calculate how much more is needed based on what is still available
        available = self.capacity - self.root.size
        return needed_space - available

    def findSmallestDelete(self, needed_space):
        # get the smallest directory which can be deleted to meet the requirements
        more_needed = self.findMoreNeeded(needed_space)
        return self.root.findSmallest(more_needed, smallest=math.inf)


### Answers

In [78]:
cap = 70000000
need = 30000000

fs = fileSystem(capacity=cap)
fs.buildTree(data)
# Part 1
print('Sum of directories under 100,000 in size: ' + str(fs.getUnder100k()))
# Part 2
print('Size of the smallest directory which can be deleted to make space: '
      + str(fs.findSmallestDelete(need)))

Sum of directories under 100,000 in size: 1783610
Size of the smallest directory which can be deleted to make space: 4370655
